# Fantasy Basketball

##### Projecting FG%, FT%, 3PM, Points, Rebounds, Assists, Steals, Blocks and Turnovers based on:
* historical performance in these categories
* team characteristics (pace, W/L, etc.)
* minutes that the player will get (may depend on some subjective factors and who else may be playing on their team)
* injuries (games played)

##### Data that we will need to scrape:
* player data for each season
* team data for each season
* coach data for each season

### Team Data Scraping

In [1]:
# teams are identified by a unique 3 letter string
# two tables: current and defunct teams

In [2]:
import requests
import re
import pandas as pd
import string
from bs4 import BeautifulSoup

In [3]:
base_url = 'https://www.basketball-reference.com'

In [25]:
def get_team_links():
    link = '/teams/'
    fullpath = base_url + link
    response = requests.get(fullpath)
    if response.status_code == 200:
        page_text = response.text
        team_links = []
        soup = BeautifulSoup(page_text, "lxml")
        rows = soup.find_all("th", {"data-stat" : "franch_name"})
        for row in rows:
            tag = row.find('a')
            if tag:
                link = tag['href']
                team_links.append(link)
        print('Team List Downloaded Successfully: {}'.format(fullpath))
        return team_links
    else:
        print('Team List Download Failed: {}'.format(fullpath))
        return None

In [27]:
def parse_team_seasons(page_text):
    soup = BeautifulSoup(page_text, "lxml")
    # find links
    initials = soup.find('link', rel="canonical")['href'].split('/')[-2]
    table = soup.find('table', id=initials)
    rows = table.find_all('tr')
    # initialize the DF for season stats
    df = pd.DataFrame()
    #first row headers
    for row in rows[1:]:
        row_df = pd.DataFrame() # temp new dataframe
        th = row.find('th')
        canonical = th.find('a')['href']
        row_df['canonical'] = [canonical]
        header = th['data-stat']
        value = th.text
        row_df[header] = [value]
        tds = row.find_all('td')
        initials = canonical.split('/')[-2]
        row_df['initials'] = [initials]
        for td in tds:
            header = td['data-stat']
            value = td.text
            row_df[header] = [value]
        df = df.append(row_df)
    df.reset_index(drop=True, inplace=True)
    return df

In [28]:
team_seasons_df = pd.DataFrame()
paths = get_team_links()
player_fails = []
print('=== DOWNLOADING TEAMS ===')
for path in paths:
    fullpath = base_url + path
    response = requests.get(fullpath)
    if response.status_code == 200:
        new_df = parse_team_seasons(response.text)
        team_seasons_df = team_seasons_df.append(new_df)
        print('Team Seasons Downloaded Successfully: {}'.format(fullpath))
    else:
        print('Team Seasons Download Failed: {}'.format(fullpath))
        player_fails.append(fullpath)
team_seasons_df.reset_index(drop=True, inplace=True)
print('=== DOWNLOADING TEAMS COMPLETED! ===')
team_seasons_df

Team List Downloaded Successfully: https://www.basketball-reference.com/teams/
=== DOWNLOADING TEAMS ===
Team Seasons Downloaded Successfully: https://www.basketball-reference.com/teams/ATL/
Team Seasons Downloaded Successfully: https://www.basketball-reference.com/teams/BOS/
Team Seasons Downloaded Successfully: https://www.basketball-reference.com/teams/NJN/
Team Seasons Downloaded Successfully: https://www.basketball-reference.com/teams/CHA/
Team Seasons Downloaded Successfully: https://www.basketball-reference.com/teams/CHI/
Team Seasons Downloaded Successfully: https://www.basketball-reference.com/teams/CLE/
Team Seasons Downloaded Successfully: https://www.basketball-reference.com/teams/DAL/
Team Seasons Downloaded Successfully: https://www.basketball-reference.com/teams/DEN/
Team Seasons Downloaded Successfully: https://www.basketball-reference.com/teams/DET/
Team Seasons Downloaded Successfully: https://www.basketball-reference.com/teams/GSW/
Team Seasons Downloaded Successfull

,canonical,season,initials,lg_id,team_name,wins,losses,win_loss_pct,rank_team,srs,pace,pace_rel,off_rtg,off_rtg_rel,def_rtg,def_rtg_rel,rank_team_playoffs,coaches,top_ws
0,/teams/ATL/2018.html,2017-18,ATL,NBA,Atlanta Hawks,,,,,,,,,,,,,,
1,/teams/ATL/2017.html,2016-17,ATL,NBA,Atlanta Hawks*,43,39,.524,2,-1.23,97.4,1.0,104.9,-3.9,105.7,-3.1,Lost E. Conf. 1st Rnd.,M. Budenholzer (43-39),D. Howard (8.3)
2,/teams/ATL/2016.html,2015-16,ATL,NBA,Atlanta Hawks*,48,34,.585,2,3.49,97.1,1.3,105.1,-1.3,101.4,-5.0,Lost E. Conf. Semis,M. Budenholzer (48-34),P. Millsap (10.1)
3,/teams/ATL/2015.html,2014-15,ATL,NBA,Atlanta Hawks*,60,22,.732,1,4.75,93.9,0.0,108.9,3.3,103.1,-2.5,Lost E. Conf. Finals,M. Budenholzer (60-22),A. Horford (8.7)
4,/teams/ATL/2014.html,2013-14,ATL,NBA,Atlanta Hawks*,38,44,.463,4,-0.88,94.6,0.7,105.9,-0.8,106.4,-0.3,Lost E. Conf. 1st Rnd.,M. Budenholzer (38-44),P. Millsap (6.7)
5,/teams/ATL/2013.html,2012-13,ATL,NBA,Atlanta Hawks*,44,38,.537,2,-0.08,92.6,0.6,104.8,-1.1,104.4,-1.5,Lost E. Conf. 1st Rnd.,L. Drew (44-38),A. Horford (8.8)
6,/teams/ATL/2012.html,2011-12,ATL,NBA,Atlanta Hawks*,40,26,.606,2,2.67,90.2,-1.1,104.9,0.3,101.2,-3.4,Lost E. Conf. 1st Rnd.,L. Drew (40-26),J. Smith (6.8)
7,/teams/ATL/2011.html,2010-11,ATL,NBA,Atlanta Hawks*,44,38,.537,3,-1.10,89.3,-2.8,106.1,-1.2,107.0,-0.3,Lost E. Conf. Semis,L. Drew (44-38),A. Horford (10.1)
8,/teams/ATL/2010.html,2009-10,ATL,NBA,Atlanta Hawks*,53,29,.646,2,4.44,90.1,-2.6,111.9,4.3,106.7,-0.9,Lost E. Conf. Semis,M. Woodson (53-29),A. Horford (10.9)
9,/teams/ATL/2009.html,2008-09,ATL,NBA,Atlanta Hawks*,47,35,.573,2,1.70,89.6,-2.1,109.3,1.0,107.6,-0.7,Lost E. Conf. Semis,M. Woodson (47-35),J. Johnson (7.6)


In [29]:
# write to file
team_seasons_df.to_csv('team_seasons_list.csv')

In [22]:
def parse_team_roster(page_text, initial, season):
    df = pd.DataFrame()
    soup = BeautifulSoup(page_text, "lxml")
    table = soup.find('table', id="roster")
    rows = table.find_all('tr')
    for row in rows[1:]:
        row_df = pd.DataFrame() # temp new dataframe
        td_first = row.find('td')
        canonical = td_first['data-append-csv']
        row_df['initial'] = [initial]
        row_df['season'] = [season]
        row_df['player_canonical'] = [canonical]
        th = row.find('th')
        header = th['data-stat']
        value = th.text
        row_df[header] = [value]
        tds = row.find_all('td')
        for td in tds:
            header = td['data-stat']
            value = td.text
            row_df[header] = [value]
        df = df.append(row_df)
    df.reset_index(drop=True, inplace=True)
    return df

In [23]:
team_rosters_df = pd.DataFrame()
roster_links = team_seasons_df['canonical'].tolist()
roster_fails = []
print('=== DOWNLOADING TEAM ROSTERS ===')
for link in roster_links:
    fullpath = base_url + link
    response = requests.get(fullpath)
    if response.status_code == 200:
        initial = link.split('/')[-2]
        season = link.split('/')[-1].replace('.html','')
        new_df = parse_team_roster(response.text, initial, season)
        team_rosters_df = team_rosters_df.append(new_df)
        print('Team Roster Downloaded Successfully: {}'.format(fullpath))
    else:
        print('Team Roster Download Failed: {}'.format(fullpath))
        roster_fails.append(fullpath)
team_rosters_df.reset_index(drop=True, inplace=True)
print('=== DOWNLOADING TEAM ROSTERS COMPLETE ===')
team_rosters_df

=== DOWNLOADING TEAM ROSTERS ===
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/ATL/2018.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/ATL/2017.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/ATL/2016.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/ATL/2015.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/ATL/2014.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/ATL/2013.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/ATL/2012.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/ATL/2011.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/ATL/2010.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/ATL/2009.html
Team Roster Downloaded Succ

Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/BOS/2000.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/BOS/1999.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/BOS/1998.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/BOS/1997.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/BOS/1996.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/BOS/1995.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/BOS/1994.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/BOS/1993.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/BOS/1992.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/BOS/1991.html
Team Roster Downloaded Successfully: https://www.basketball-

Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/NJN/1984.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/NJN/1983.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/NJN/1982.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/NJN/1981.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/NJN/1980.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/NJN/1979.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/NJN/1978.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/NYN/1977.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/NYA/1976.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/NYA/1975.html
Team Roster Downloaded Successfully: https://www.basketball-

Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/CHI/1975.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/CHI/1974.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/CHI/1973.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/CHI/1972.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/CHI/1971.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/CHI/1970.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/CHI/1969.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/CHI/1968.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/CHI/1967.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/CLE/2018.html
Team Roster Downloaded Successfully: https://www.basketball-

Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/DAL/1987.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/DAL/1986.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/DAL/1985.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/DAL/1984.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/DAL/1983.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/DAL/1982.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/DAL/1981.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/DEN/2018.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/DEN/2017.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/DEN/2016.html
Team Roster Downloaded Successfully: https://www.basketball-

...

Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/POR/1977.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/POR/1976.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/POR/1975.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/POR/1974.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/POR/1973.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/POR/1972.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/POR/1971.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/SAC/2018.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/SAC/2017.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/SAC/2016.html
Team Roster Downloaded Successfully: https://www.basketball-

Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/SAS/2007.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/SAS/2006.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/SAS/2005.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/SAS/2004.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/SAS/2003.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/SAS/2002.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/SAS/2001.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/SAS/2000.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/SAS/1999.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/SAS/1998.html
Team Roster Downloaded Successfully: https://www.basketball-

Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/UTA/1993.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/UTA/1992.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/UTA/1991.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/UTA/1990.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/UTA/1989.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/UTA/1988.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/UTA/1987.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/UTA/1986.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/UTA/1985.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/UTA/1984.html
Team Roster Downloaded Successfully: https://www.basketball-

Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/CHS/1947.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/CLR/1947.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/DNN/1950.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/DTF/1947.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/INJ/1949.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/INO/1953.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/INO/1952.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/INO/1951.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/INO/1950.html
Team Roster Downloaded Successfully: https://www.basketball-reference.com/teams/KEN/1976.html
Team Roster Downloaded Successfully: https://www.basketball-

,initial,season,player_canonical,number,player,pos,height,weight,birth_date,birth_country,years_experience,college_name
0,ATL,2018,schrode01,,Dennis Schroder,G,6-1,172,"September 15, 1993",de,4,
1,ATL,2018,belinma01,,Marco Belinelli,G-F,6-5,210,"March 25, 1986",it,10,
2,ATL,2018,bazemke01,,Kent Bazemore,G-F,6-5,201,"July 1, 1989",us,5,Old Dominion University
3,ATL,2018,delanma01,,Malcolm Delaney,G,6-3,190,"March 11, 1989",us,1,Virginia Polytechnic Institute and State Unive...
4,ATL,2018,princta02,,Taurean Prince,F,6-8,220,"March 22, 1994",us,1,Baylor University
5,ATL,2018,bembrde01,,DeAndre' Bembry,F,6-6,210,"July 4, 1994",us,1,Saint Joseph's University
6,ATL,2018,brussni01,,Nicolas Brussino,G-F,6-7,195,"March 2, 1993",ar,1,
7,ATL,2018,plumlmi01,,Miles Plumlee,C-F,6-11,249,"September 1, 1988",us,5,Duke University
8,ATL,2018,collijo01,,John Collins,F,6-10,235,"September 23, 1997",us,R,Wake Forest University
9,ATL,2018,dorsety01,,Tyler Dorsey,G,6-5,183,"February 14, 1996",us,R,University of Oregon


In [24]:
# write to file
team_rosters_df.to_csv('team_rosters_list.csv')